In [3]:
import random

import numpy as np
from matplotlib import pyplot as plt
import math
import time

Array Algorithms

In [30]:
#Sorts arr in ascending order. Hell nah don't use. array.sort() is substantially faster. Python moment
def mergeSort(arr, l, r):
    def merge(arr, l, m, r):
        n1 = m - l + 1
        n2 = r - m

        # create temp arrays
        L = [0] * (n1)
        R = [0] * (n2)

        # Copy data to temp arrays L[] and R[]
        for i in range(0, n1):
            L[i] = arr[l + i]

        for j in range(0, n2):
            R[j] = arr[m + 1 + j]

        # Merge the temp arrays back into arr[l..r]
        i = 0  # Initial index of first subarray
        j = 0  # Initial index of second subarray
        k = l  # Initial index of merged subarray
        while i < n1 and j < n2:
            if L[i] <= R[j]:
                arr[k] = L[i]
                i += 1
            else:
                arr[k] = R[j]
                j += 1
            k += 1
        # Copy the remaining elements of L[], if there
        # are any
        while i < n1:
            arr[k] = L[i]
            i += 1
            k += 1
        # Copy the remaining elements of R[], if there
        # are any
        while j < n2:
            arr[k] = R[j]
            j += 1
            k += 1

    if l < r:
        m = (l+r)//2

        # Sort first and second halves
        mergeSort(arr, l, m)
        mergeSort(arr, m + 1, r)
        merge(arr, l, m, r)

#Find an element in sorted arr that is less or equal to k
def binarySearch(arr, k):
    def binaryRec(l, r):
        if(l > r):
            return -1
        m = (l + r) // 2
        if k == arr[m]:
            return m
        if k < arr[m]:
            return binaryRec(l, m - 1)
        else:
            return binaryRec(m + 1, r)
    ans = binaryRec(0, len(arr) - 1)
    if arr[ans] != k:
        return -1
    return ans
#Find an element in sorted arr that is less or equal to k
def binarySearchLEQ(arr, k):
    def binaryRec(l, r):
        if(l == r):
            return l
        m = math.ceil((l + r) / 2)
        if k < arr[m]:
            return binaryRec(l, m - 1)
        else:
            return binaryRec(m, r)
    ans = binaryRec(0, len(arr) - 1)
    if arr[ans] == k:
        if arr[ans - 1] <= k:
            return arr[ans - 1]
        return -1
    if arr[ans] <= k:
        return arr[ans]
    return -1

arr = np.random.randint(0, 40000000, size = 10000000)
finding = arr[69]
sortT = time.time()
arr.sort()
# mergeSort(arr, 0, len(arr) - 1)
sortT = time.time() - sortT
st = time.time()
found = binarySearch(arr, finding)
display(str(arr[found]) + " found in time: " + str(time.time() - st))
display(str(finding) + " was the required element, time spent sorting: " + str(sortT))

'37818624 found in time: 0.0'

'37818624 was the required element, time spent sorting: 114.316899061203'

K-D Tree Along With Nearest NeighbourSearch

In [ ]:
class Node:
    def __init__(self, point):
        self.point = point
        self.left = None
        self.right = None

# n - number of dimensions
def kdTreeConstuct(n, arr):
    def areSame(point1, point2):
        for i in range(0, len(point1)):
            if np.isclose(point1[i], point2[i]) == False:
                return False
        return True

    def insertPoint(root: Node, point: np.array, depth):
        if not root:
            return Node(point)
        if areSame(root.point, point) == True:
            return root
        if point[depth % n] >= root.point[depth % n]:
            root.right = insertPoint(root.right, point, depth + 1)
        else:
            root.left = insertPoint(root.left, point, depth + 1)

        return root

    root: Node = Node(arr[0])
    for i in range(1, len(arr)):
        insertPoint(root, arr[i], 0)
    return root


def getSQdist(point1, point2):
    dist = 0
    for i in range(0, len(point1)):
        dist = dist + ((point2[i] - point1[i]) ** 2)
    return dist

# n - number of dimensions
def FindNeighbour(root: Node, target, n):
    currentBestPoint = None
    currentBestDistance = math.inf

    def recurDown(root: Node, target, depth):
        nonlocal currentBestPoint
        nonlocal currentBestDistance
        if not root:
            return math.inf
        c_dm = depth % n
        if (target[c_dm] >= root.point[c_dm]):
            recurDown(root.right, target, depth + 1)
        else:
            recurDown(root.left, target, depth + 1)

        dist = getSQdist(root.point, target)
        if dist < currentBestDistance:
            currentBestPoint = root.point
            currentBestDistance = dist

        # check if there can possibly be better options on the other side of the dividing line
        if ((root.point[c_dm] - target[c_dm]) ** 2) > currentBestDistance:
            return
        # Do the same thing for the sibling branch
        if (target[c_dm] >= root.point[c_dm]):
            recurDown(root.left, target, depth + 1)
        else:
            recurDown(root.right, target, depth + 1)

    recurDown(root, target, 0)
    return currentBestPoint

# works exactly the same as ordinary NN, just storing the best k neighbours in stack
# n = number of dimensions; k_neighbours - number of neighbours
def FindKNN(root: Node, target, n, k_neighbours=1):
    currentBestDistance = np.full(k_neighbours, math.inf)
    currentBestPoint = np.full(k_neighbours, None)
    #Adds the new point to a stack. [0] - closest point; [k - 1] - furthest point(from best)
    def addNewDist(dist, point):
        nonlocal currentBestDistance
        nonlocal currentBestPoint
        for i in reversed(range(0, k_neighbours)):
            if (dist < currentBestDistance[i]):
                if(i != k_neighbours - 1):
                    currentBestDistance[i + 1] = currentBestDistance[i]
                    currentBestPoint[i + 1] = currentBestPoint[i]
                if(i == 0):
                    currentBestDistance[0] = dist
                    currentBestPoint[0] = point
                    break
            else:
                currentBestDistance[i + 1] = dist
                currentBestPoint[i + 1] = point
                break

    def recurDown(root: Node, target, depth):
        if not root:
            return math.inf
        cd = depth % n
        if(target[cd] >= root.point[cd]):
            recurDown(root.right, target, depth + 1)
        else:
            recurDown(root.left, target, depth + 1)

        dist = getSQdist(root.point, target)
        if(dist < currentBestDistance[k_neighbours - 1]):
            addNewDist(dist, root.point)

        #Litterally the only place where algorithm differs, now it checks if furthest neighbour could possibly be replaced
        if ((root.point[cd] - target[cd])**2) >= currentBestDistance[k_neighbours - 1]:
            return
        if(target[cd] >= root.point[cd]):
            recurDown(root.left, target, depth + 1)
        else:
            recurDown(root.right, target, depth + 1)

    recurDown(root, target, 0)
    return currentBestPoint[0:k_neighbours]

#Driver Code
points = np.random.randint(50, size=(150, 2))
plt.figure(figsize=(12,12))
plt.grid(True)
plt.xticks(np.arange(min(points[:,0]), max(points[:,0])+1, 1))
plt.yticks(np.arange(min(points[:,1]), max(points[:,1])+1, 1))
plt.plot(points[:,0], points[:,1], 'o')
target = np.random.randint(5,45, size=2)
plt.plot(target[0], target[1],'x',color='orange')
root = kdTreeConstuct(2, points)
closest = FindNeighbour(root, target, 2)
plt.plot(closest[0], closest[1],'o',color='red')
# b = 0
# for i in closest:
#     plt.plot(i[0], i[1],'o',color='red')
#     plt.annotate(b, xy =(i[0],i[1]),
#                 xytext =(i[0],i[1]))
#     b += 1

Binary Tree

In [ ]:
# construct a Binary Search Tree from array
def binarySearchTreeConstruct(arr):

    def insertPoint(root:Node, point:int):
        if not root:
            return Node(point)
        if np.isclose(root.point, point) == True:
            return root
        if point >= root.point:
            root.right = insertPoint(root.right, point)
        else:
            root.left = insertPoint(root.left, point)

        return root
    root: Node = Node(arr[0])
    for i in range(1, len(arr)):
        insertPoint(root, arr[i])
    return root

def traverseTreeInOrder(root:Node):
    traversal = []
    def recurDown(root: Node):
        nonlocal traversal
        if not root:
            return
        recurDown(root.left)
        traversal.append(root.point)
        recurDown(root.right)

    recurDown(root)
    return traversal

def traverseTreePreOrder(root:Node):
    traversal = []
    def recurDown(root: Node):
        nonlocal traversal
        if not root:
            return
        traversal.append(root.point)
        recurDown(root.left)
        recurDown(root.right)

    recurDown(root)
    return traversal

def traverseTreePostOrder(root:Node):
    traversal = []
    def recurDown(root: Node):
        nonlocal traversal
        if not root:
            return
        recurDown(root.left)
        recurDown(root.right)
        traversal.append(root.point)

    recurDown(root)
    return traversal

# construct a binary tree from its InOrder and PreOrder traversals and return its root
def constructTreeFromTraversals(inorder, preorder, n):
    currentPreorderIndex = 0
    #startInd, endInd - inclusive borders of subTree in InOrder array
    def buildTreeRec(startInd, endInd, inorder, preorder):
        nonlocal currentPreorderIndex
        if (startInd > endInd):
            return None
        root = Node(preorder[currentPreorderIndex])

        rootIndex = 0#index of root of the subTree in InOrder array
        for i in range(startInd, endInd + 1):
            if(inorder[i] == root.point):
                rootIndex = i
                break
        currentPreorderIndex += 1
        if (startInd == endInd):
            return root
        root.left = buildTreeRec(startInd, rootIndex - 1, inorder, preorder)
        root.right = buildTreeRec(rootIndex + 1, endInd, inorder, preorder)
        return root

    currentPreorderIndex = 0
    return buildTreeRec(0, n - 1, inorder, preorder)

# gets the size of largest Binary Search subTree inside the Binary Tree(given by root)
def findMaxBST(root: Node):
    def recurDown(root: Node):
        if not root:
            return [0, 1000000000, -1000000000]# [size of BST, minimum node in BST, maximum node in BST]
        if (not root.left) & (not root.right):
            return [1, root.point, root.point]
        rightData = recurDown(root.right)
        leftData = recurDown(root.left)
        #if the root is between maximum value of left subBST and minimum of right subBST, then all 3 form BST as well
        if (rightData[1] > root.point) and (leftData[2] < root.point):
            if rightData[0] <= 0:
                return [rightData[0] + leftData[0] + 1, leftData[1], root.point]
            if leftData[0] <= 0:
                return [rightData[0] + leftData[0] + 1, root.point, rightData[2]]

            return [rightData[0] + leftData[0] + 1, leftData[1], rightData[2]]
        else:
            return [max(rightData[0], leftData[0]), -1000000000, 1000000000]
    return recurDown(root)[0]

# Construct A Binary Search Tree Given Its Preorder
def constructBSTfromPreorder_Alt(preorder):
    def recurDown(Ind, root: Node):
        if not root:
            return Node(preorder[Ind])

        if preorder[Ind] < root.point:
            root.left = recurDown(Ind, root.left)
        else:
            root.right = recurDown(Ind,root.right)

        return root
    root_ = recurDown(0, None)
    for i in range(1, len(preorder)):
        recurDown(i, root_)
    return root_